## Model making

In [1]:
import torch
import os
import transformers
from ast import literal_eval
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM, SFTConfig
from datasets import Dataset
import json
import pandas as pd
import random
import numpy as np
import evaluate
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments, DataCollatorForSeq2Seq, DataCollatorForLanguageModeling
from sklearn.utils.class_weight import compute_class_weight
from torch.nn import CrossEntropyLoss

# 1. 설정: pandas 출력 옵션 및 시드 고정
pd.set_option('display.max_columns', None)
os.environ["TOKENIZERS_PARALLELISM"] = "false"
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(random_seed)
    random.seed(random_seed)

set_seed(42)
# 2. 모델 및 토크나이저 로드 (8-bit 양자화)
model_name = "CarrotAI/Llama-3.2-Rabbit-Ko-3B-Instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # QLoRA는 4bit 양자화를 사용
    # load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,  # 계산 precision (float16 또는 bfloat16 사용 가능)
    bnb_4bit_use_double_quant=True,       # 이중 양자화 활성화
    bnb_4bit_quant_type="nf4"             # NF4 양자화 타입
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # BitsAndBytesConfig 추가
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"  


# 3. PEFT 설정 (LoRA)
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.05,
    target_modules=['q_proj', 'v_proj', 'k_proj', 'o_proj'],
    bias="none",
    task_type="CAUSAL_LM",
)

# model = get_peft_model(model, lora_config)


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.02s/it]


## Data Processing

In [2]:
def make_dataset(dataset, prompts, system_prompt):
    records = []
    for _, row in dataset.iterrows():
        problems = literal_eval(row['problems'])
        record = {
            'id': row['id'],
            'paragraph': row['paragraph'],
            'question': problems['question'],
            'choices': problems['choices'],
            'answer': problems.get('answer', None),
            "question_plus": problems.get('question_plus', ''),
            'klue' : row.get('klue', None),
            'question_type' : row.get('question_type', None)
        }
        records.append(record)
            
    # Convert to DataFrame
    df = pd.DataFrame(records)
    dataset = Dataset.from_pandas(df)
    processed_dataset = []

    for i in range(len(dataset)):
        choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(dataset[i]["choices"])])
        if dataset[i]['question_type'] == '이해형':
            prompt = prompts.이해형
        elif dataset[i]['question_type'] == '기타':
            prompt = prompts.기타
        elif dataset[i]['question_type'] == '사실형':
            prompt = prompts.사실형
        elif dataset[i]['question_type'] == '추론형':
            prompt = prompts.추론형
        else:
            prompt = prompts.나열형
        user_message = prompt.format(
            paragraph=dataset[i]["paragraph"],
            question=dataset[i]["question"],
            question_plus=dataset[i]["question_plus"],
            question_type=dataset[i]['question_type'],
            choices=choices_string,
        )
        # chat message 형식으로 변환
        processed_dataset.append(
            {
                "id": dataset[i]["id"],
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message},
                    {"role": "assistant", "content": f"{dataset[i]['klue']}"}
                ],
                "label": dataset[i]['answer'],
                'type' : dataset[i]['question_type']
            }
        )
    processed_dataset = Dataset.from_pandas(pd.DataFrame(processed_dataset))

    def formatting_prompts_func(example):
        output_texts = []
        for i in range(len(example["messages"])):
            output_texts.append(
                tokenizer.apply_chat_template(
                    example["messages"][i],
                    tokenize=False,
                )
            )
        return output_texts

    def tokenize(element):
        outputs = tokenizer(
            formatting_prompts_func(element),
            truncation=False,
            padding=False,
            return_overflowing_tokens=False,
            return_length=False,
        )
        return {
            "input_ids": outputs["input_ids"],
            "attention_mask": outputs["attention_mask"],
        }

    # 데이터 토큰화
    tokenized_dataset = processed_dataset.map(
        tokenize,

        batched=True,
        num_proc=1,
        load_from_cache_file=True,
        desc="Tokenizing",
    )

    # 데이터 분리
    tokenized_dataset = tokenized_dataset.filter(lambda x: len(x["input_ids"]) <= 2048)  
    tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.10, seed=42)

    train_dataset = tokenized_dataset['train']
    eval_dataset = tokenized_dataset['test']


    train_dataset_token_lengths = [len(train_dataset[i]["input_ids"]) for i in range(len(train_dataset))]
    print(f"max token length: {max(train_dataset_token_lengths)}")
    print(f"min token length: {min(train_dataset_token_lengths)}")
    print(f"avg token length: {np.mean(train_dataset_token_lengths)}")

    # 데이터 확인
    return train_dataset, eval_dataset

In [3]:
from prompts import user_prompts
system_prompt = """지시에 따라 주어진 문제의 정답을 구하세요."""
prompts = user_prompts
dataset = pd.read_csv('datas/train+klue.csv')
train_dataset, eval_dataset = make_dataset(dataset,prompts, system_prompt)
# 여기서 프롬프트만 바꿔서 데이터셋을 만들기

Filter: 100%|██████████| 2031/2031 [00:01<00:00, 1463.28 examples/s]


max token length: 1664
min token length: 214
avg token length: 750.4258347016968


In [4]:
print(tokenizer.decode(train_dataset['input_ids'][5]))

<|begin_of_text|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

지시에 따라 주어진 문제의 정답을 구하세요.<|eot_id|><|start_header_id|>user<|end_header_id|>

문제 유형:
기타

지문:
본문 1:
“전하, 게다가 우리들의 왕국에는 신께 도움이 되지 않는 불편함이 존재합니다. 바로 우리 백성들 중 많은 이들이 그대의 백성들이 가져오는 왕국의 상품과 물건을 간절히 원하고 있다는 것, 그런데 그대의 백성들은 자신들의 탐욕스러운 욕망을 만족시키기 위해 자유민이자 해방된 나의 백성들을 잡아가고 있다는 것입니다. 심지어 귀족과 왕의 친척까지도 잡아가 우리들의 왕국에 있는 백인들에게 팔고 있다는 것입니다.”
콩고의 아폰소 1세 국왕이 포르투갈의 주앙 3세 국왕에게 보낸 편지, 1526
출처 2:
“이번 원정에 많은 비용이 들었기에 빈 손으로 돌아간다면 합리적이지 못한 일이 될 것이다. 우리의 [주된] 바람은 신을 섬기는 것과 콩고 국왕을 기쁘게 하는 것이지만, 그럼에도 불구하고 콩고 국왕으로 하여금 노예가 됐건 구리가 됐건 상아가 됐건 배를 채워야 한다는 사실을 우리들의 이름으로 이해시켜야만 한다.”
포르투갈 마누엘 국왕의 콩고에 있는 사절에게 보낸 편지, 1512

질문:
편지에 설명된 상호 작용은 다음 중 어떤 맥락에서 가장 잘 이해되는가?

선택지:
1 - 포르투갈의 서아프리카 해안 탐험
2 - 사하라 이남 아프리카에서의 가톨릭 선교 활동
3 - 사하라 이남 아프리카의 국가 형성
4 - 사하라 이남 아프리카의 노예 무역 발전

출력 형식:
근거 : 답변을 도출한 텍스트 # 정답 번호

문제는 무조건 1개의 정답만 있습니다.
문제를 풀이할 때, 반드시 지문을 참고하세요.
반드시 지문에서 정답의 근거를 찾으세요.
반드시 출력 형식을 지키세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

근거 : 내용 

## Train phase

In [5]:
import numpy as np
import evaluate
acc_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")
def preprocess_logits_for_metrics(logits, labels):
    logits = logits if not isinstance(logits, tuple) else logits[0]
    logit_idx = [tokenizer.vocab["1"],
                    tokenizer.vocab["2"],
                    tokenizer.vocab["3"],
                    tokenizer.vocab["4"], 
                    tokenizer.vocab["5"]]
    logits = logits[:, -2, logit_idx] # -2: answer token, -1: eos token
    return logits


    # metric 계산 함수
def compute_metrics(evaluation_result):
    logits, labels = evaluation_result
    int_output_map = {"1": 0, "2": 1, "3": 2, "4": 3, "5": 4}


    # 토큰화된 레이블 디코딩
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    labels = [label.split('#')[-1].strip() for label in labels]
    labels = [int_output_map.get(label, -1) for label in labels] 

    # 소프트맥스 함수를 사용하여 로그트 변환
    probs = torch.nn.functional.softmax(torch.tensor(logits, dtype=torch.float32), dim=-1)

    predictions = np.argmax(probs, axis=-1)

    # 정확도 계산
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="macro")

    return {"accuracy": acc["accuracy"], "f1": f1["f1"]}


In [6]:
response_template = "assistant<|end_header_id|>"
data_collator = DataCollatorForCompletionOnlyLM(
    response_template=response_template,
    tokenizer=tokenizer,
)

sft_config = SFTConfig(
    do_train=True,
    do_eval=True,
    lr_scheduler_type="cosine",
    max_seq_length = 2048,
    output_dir=f"./outputs + {model_name.split('/')[-1]}",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    per_device_eval_batch_size=1,
    num_train_epochs=2,
    learning_rate=1e-5,
    weight_decay=0.01,
    logging_steps=10,
    save_strategy="epoch",
    eval_strategy="epoch",
    save_total_limit=2,
    save_only_model=True,
    report_to="none",
    fp16 = True,
    gradient_checkpointing = False, # 8B모델 돌릴때만 True로, 아니면 False로
    load_best_model_at_end = True
)
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics = preprocess_logits_for_metrics,
    args=sft_config,
    packing=False,
    peft_config = lora_config
)
torch.cuda.empty_cache()


/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:463: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a valid formatting function. Therefore `formatting_func` will be ignored.
  warnings.warn(


In [7]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,F1
0,1.156900,0.950638,0.362745,0.275730
1,1.017800,0.937394,0.357843,0.280685


[[128001 128001 128001 ... 128001 128001 128001]
 [128001 128001 128001 ... 128001 128001 128001]
 [128001 128001 128001 ... 128001 128001 128001]
 ...
 [128001 128001 128001 ... 128001 128001 128001]
 [128001 128001 128001 ... 128001 128001 128001]
 [128001 128001 128001 ... 128001 128001 128001]]
[[128001 128001 128001 ... 128001 128001 128001]
 [128001 128001 128001 ... 128001 128001 128001]
 [128001 128001 128001 ... 128001 128001 128001]
 ...
 [128001 128001 128001 ... 128001 128001 128001]
 [128001 128001 128001 ... 128001 128001 128001]
 [128001 128001 128001 ... 128001 128001 128001]]


TrainOutput(global_step=456, training_loss=1.0753026677851092, metrics={'train_runtime': 1815.1474, 'train_samples_per_second': 2.013, 'train_steps_per_second': 0.251, 'total_flos': 4.659578527995494e+16, 'train_loss': 1.0753026677851092, 'epoch': 1.9967159277504105})

## Inference Phase

In [9]:
from ast import literal_eval
from datasets import Dataset

def make_test_dataset(test_df, prompt, system_prompt):
    # Flatten the JSON dataset
    records = []
    for _, row in test_df.iterrows():
        problems = literal_eval(row['problems'])
        record = {
            'id': row['id'],
            'paragraph': row['paragraph'],
            'question': problems['question'],
            'choices': problems['choices'],
            'answer': problems.get('answer', None),
            "question_plus": problems.get('question_plus', ''),
            'klue': row.get('klue', None),
            'question_type': row.get('question_type', None)
        }
        records.append(record)

    # Convert to DataFrame
    df = pd.DataFrame(records)
    dataset = Dataset.from_pandas(df)
    
    # test_dataset를 빈 리스트로 초기화
    test_dataset = []

    # dataset의 각 항목에 대해 처리
    for i in range(len(dataset)):
        # choices_string 생성
        choices_string = "\n".join([f"{idx + 1} - {choice}" for idx, choice in enumerate(dataset[i]["choices"])])
        
        # question_type에 맞는 prompt 선택
        if dataset[i]['question_type'] == '이해형':
            prompt = prompts.이해형
        elif dataset[i]['question_type'] == '기타':
            prompt = prompts.기타
        elif dataset[i]['question_type'] == '사실형':
            prompt = prompts.사실형
        elif dataset[i]['question_type'] == '추론형':
            prompt = prompts.추론형
        else:
            prompt = prompts.나열형

        # user_message 생성
        user_message = prompt.format(
            paragraph=dataset[i]["paragraph"],
            question=dataset[i]["question"],
            question_plus=dataset[i]["question_plus"],
            question_type=dataset[i]['question_type'],
            choices=choices_string,
        )

        # len_choices는 choices의 길이로 계산
        len_choices = len(dataset[i]["choices"])

        # test_dataset에 항목 추가
        test_dataset.append(
            {
                "id": dataset[i]["id"],  # dataset[i]에서 'id' 가져오기
                "messages": [
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_message},
                ],
                "label": dataset[i].get('answer', None),  # 'answer'가 없으면 None
                'type': dataset[i].get('question_type', None),
                "len_choices": len_choices,
            }
        )

    return test_dataset


In [22]:
from torch.utils.data import DataLoader
from prompts import user_prompts

test_df = pd.read_csv('datas/test_processing_4.csv')
prompts = user_prompts
test_dataset = make_test_dataset(test_df, prompts, system_prompt)


In [25]:
test_dataset[130]

{'id': 'generation-for-nlp-130',
 'messages': [{'role': 'system', 'content': '지시에 따라 주어진 문제의 정답을 구하세요.'},
  {'role': 'user',
   'content': '\n문제 유형:\n사실형\n\n지문:\n한  평도  채  안  되는  구 멍가게 는 중풍으로 쓰러져 정상적 건강   상태가 아니었던 아버지의 유일한 수입원이자 생존 이유였다.   때문에 ㉠그 구멍가게에 대한 아버지의 몰두와 자존심은 각별했다 . 한번은 내가 아버지가 가게를 잠깐 비운 사이에 겉에 허연  인공 설탕 가루를 묻힌 ‘미키대장군’ 이라는 캐러멜을 하나 아무  생각 없이 널름 집어먹은 적이 있었다.  하나에 이 원 ,  다섯 개에   십 원이었다.  잠시 뒤에 돌아온 아버지는 단박에 그 사실을 알아 채고는 불같이 화를 내며 내 목덜미에 당수를 한 대 세게 내려 꽂는 것이었다.  그 캐러멜 갑 안에 미키대장군이 몇 개 들어  있는지조차 훤히 꿰차고 있는 아버지였다 . ―이런 민한 종간나래 !  얌생이처럼 기러케 쏠라닥질을 허자면   이 가게 안에 뭐이가 하나 제대로 남아나겠니 ,  응 ? 그러고 나서는 좀 머쓱했는지 입이 한 발쯤 튀어나와 뾰로통 해서 서 있는 내게 미키대장군 네 개를 집어 내미는 거였다 .   어차피 짝이 맞아야 파니까니,  하면서 억지로 내 손아귀에 쥐어 주었다.  ㉡나는 그 무허가 불량 식품인 캐러멜 네 개가 끈끈하게   녹아내릴 때까지 먹지 않고 쥔 채 서 있었다 . ―늴큼 털어 넣지 못하겠니,  으잉? 목덜미에 아버지의 가벼운 당수를 한 대 더 얹은 다음에야  한입에 털어 넣고 돌아서 나왔다.  아버지도 가게 일을 수월하게   보려면 잔심부름꾼인 나를 무시하고는 아쉬울 때가 많을 터였다 .   워낙 짧은 밑천으로 가게를 꾸려 가자니 아버지는 물건 구색을  맞추느라 하루에도 많을 때는 세 번까지 시장통 도매상으로 정부미   포대를 거머쥐고 종종걸음을 쳐야 했고 ,  막내인 나는 번번이 

In [26]:
tokenizer.padding_side = 'left'


# 배치 데이터 로더를 위한 collate_fn
def collate_fn(batch):
    ids = [item["id"] for item in batch]
    messages = [item["messages"] for item in batch]
    labels = [item.get("label", None) for item in batch]  # 라벨이 존재할 경우만 가져옴
    return ids, messages, labels

## 1. Test dataset inference


In [27]:
# 데이터 로더 설정
batch_size = 8  # 배치 크기 설정
dataloader = DataLoader(test_dataset, batch_size=batch_size, collate_fn=collate_fn)

generated_infer_results = []

with torch.inference_mode():
    for batch in tqdm(dataloader):
        ids, messages, labels = batch

        # 텍스트 생성을 위한 입력 데이터 준비
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,  # 배치 크기에 맞게 패딩 추가
        )

        # GPU로 이동 (inputs가 Tensor일 경우 바로 이동)
        inputs = inputs.to(model.device)  # Tensor로 직접 처리

        # 모델 생성
        outputs = model.generate(
            inputs,
            max_new_tokens=150,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
        )

        # 결과 디코딩
        generated_texts = tokenizer.batch_decode(
            outputs[:, inputs.shape[1]:], skip_special_tokens=True
        )

        # 결과 저장
        for _id, generated_text, label in zip(ids, generated_texts, labels):
            generated_infer_results.append({
                "id": _id,
                "answer": generated_text,
                "label": label  # 실제 라벨이 있다면 포함
            })

# 결과를 DataFrame으로 저장
generated_infer_results = pd.DataFrame(generated_infer_results)

  6%|▌         | 6/109 [02:08<33:29, 19.51s/it]

In [28]:
generated_infer_results

,id,answer,label
0,generation-for-nlp-0,"근거 : 내용에 따르면, 독자는 책의 내용을 통해 자신의 사회나 시대의 영향을 경험...",
1,generation-for-nlp-1,"근거 : 내용에 따르면, 독자는 책과 소통하는 즐거움을 경험하며, 자신이 속한 사회...",
2,generation-for-nlp-2,"근거 : 내용에 따르면, (가)와 (나)에서 유서의 유형과 분류, 편찬 방식, 국가...",
3,generation-for-nlp-3,"근거 : 내용에 따르면, 조선에서 편찬자가 미상인 유서가 많았던 것은 편찬자의 개인...",
4,generation-for-nlp-4,"근거 : 내용에 따르면 ㉠은 서학의 세부 내용을 다른 분야로 확대하고, 상호 참조하...",
5,generation-for-nlp-5,"근거 : 내용에 따르면, 이수광은 서양 학문과 주자학을 적극적으로 수용하였으며, 서...",
6,generation-for-nlp-6,"근거 : 내용에 따르면, (가)와 (나)에서 유서 편찬 경향과 실학자들의 역할이 명...",
7,generation-for-nlp-7,"근거 : 내용에 따르면, 문맥상 ⓐ ～ⓔ의 바꾸는 것은 서양 학문이 조선 유서에 미...",
8,generation-for-nlp-8,"근거 : 내용에 따르면, 불확정 개념은 법조문과 행정 법령에서 사용되며, 구체적 상...",
9,generation-for-nlp-9,"근거 : 내용에 따르면, ㉠은 재량 준칙으로 정해진 행정 작용의 기준을 명확히 정할...",


In [29]:
import re
def extract_last_digit(s):
    match = re.search(r'\d$', s)  # 문자열 끝에서 숫자 하나만 매칭
    return match.group() if match else None  # 매칭된 숫자를 반환, 없으면 None 반환

result = generated_infer_results
# 데이터프레임에 적용
result['text'] = result['answer']
result['answer'] = result['answer'].apply(lambda x: x.split('#')[-1])
result['answer'] = result['answer'].apply(extract_last_digit)
result['answer'] = result['answer'].fillna(1)
result['answer'] = result['answer'].apply(lambda x: int(x))
submission = result[['id', 'text', 'answer']]
submission['answer'].value_counts()

answer
4    7
3    5
5    2
1    2
Name: count, dtype: int64

### 2. Eval dataset으로 inference
#### 프롬프트를 바꿔가며 몇개 맞추나 보기

In [13]:
tokenizer.padding_side = 'left'
batch_size = 8  # 배치 크기 설정
dataloader = DataLoader(eval_dataset, batch_size=batch_size, collate_fn=collate_fn)

generated_infer_results = []

with torch.inference_mode():
    for batch in tqdm(dataloader):
        ids, messages, labels = batch

        # 텍스트 생성을 위한 입력 데이터 준비
        inputs = tokenizer.apply_chat_template(
            messages,
            tokenize=True,
            add_generation_prompt=True,
            return_tensors="pt",
            padding=True,  # 배치 크기에 맞게 패딩 추가
        )

        # GPU로 이동 (inputs가 Tensor일 경우 바로 이동)
        inputs = inputs.to(model.device)  # Tensor로 직접 처리

        # 모델 생성
        outputs = model.generate(
            inputs,
            max_new_tokens=150,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,  # 종료 토큰 설정
        )

        # 결과 디코딩
        generated_texts = tokenizer.batch_decode(
            outputs[:, inputs.shape[1]:], skip_special_tokens=True
        )
        

        # 결과 저장
        for _id, generated_text, label in zip(ids, generated_texts, labels):
            generated_infer_results.append({
                "id": _id,
                "answer": generated_text,
                "label": label  # 실제 라벨이 있다면 포함
            })

# 결과를 DataFrame으로 저장
generated_infer_results = pd.DataFrame(generated_infer_results)

100%|██████████| 26/26 [06:20<00:00, 14.62s/it]


In [16]:
from copy import deepcopy

import re
def extract_last_digit(s):
    match = re.search(r'\d$', s)  # 문자열 끝에서 숫자 하나만 매칭
    return match.group() if match else None  # 매칭된 숫자를 반환, 없으면 None 반환

result = deepcopy(generated_infer_results)
# 데이터프레임에 적용
result['text'] = result['answer']
result['predict'] = result['answer'].apply(lambda x: x.split('#')[-1])
result['predict'] = result['predict'].apply(extract_last_digit)
result['predict'] = result['predict'].fillna(1)
result['predict'] = result['predict'].apply(lambda x: int(x))

result = result[['text', 'predict', 'label',]]
result['question_type'] = eval_dataset['type']
result['맞았는지 여부'] = result['predict'] == result['label']


In [19]:
result['맞았는지 여부'].value_counts()

맞았는지 여부
True     176
False     28
Name: count, dtype: int64